Sources for the code:  
https://omes-va.com/mediapipe-hands-python/  
https://omes-va.com/contando-dedos-mediapipe-opencv-python/  
https://www.toolify.ai/es/ai-news-es/domina-la-deteccin-de-manos-y-la-estimacin-de-posturas-con-mediapipe-443934  


In [1]:
import cv2
import mediapipe as mp
import numpy as np

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

video_path = "hand_video.mp4"
#video_path = "FroggerHighway.mp4"
cap = cv2.VideoCapture(video_path)

# properties from original video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# VideoWriter object to save the video
fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
out = cv2.VideoWriter('output_hand_video.mp4', fourcc, fps, (frame_width, frame_height))

paint_points= np.empty((0, 2), dtype=np.int32)
while True:
    ret, frame = cap.read()
    if not ret:
        break

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    index_tip=None
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # coordinates from index finger
            index_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP] #Represents the tip of the index finger
            index_pip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_PIP] #Corresponds to the proximal interphalangeal joint.
            index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP] #Indicates the metacarpophalangeal joint.

            # Normalize coordinates into pixels
            h, w, _ = frame.shape
            tip_x, tip_y = int(index_tip.x * w), int(index_tip.y * h)
            pip_x, pip_y = int(index_pip.x * w), int(index_pip.y * h)
            mcp_x, mcp_y = int(index_mcp.x * w), int(index_mcp.y * h)
          
            tmp_point = np.array([[tip_x, tip_y]], dtype=np.int32)
            paint_points= np.append(paint_points,tmp_point,axis=0)
            #Draw blue line to follow the index tip
            cv2.polylines(frame, [paint_points], isClosed=False, color=(255,0,0), thickness=2)
            # set the points from the index finger 
            finger_width = 23  # standard width
            points = np.array([
                [mcp_x - finger_width, mcp_y],
                [pip_x - finger_width, pip_y],
                [tip_x - finger_width // 2, tip_y],
                [tip_x + finger_width // 2, tip_y],
                [pip_x + finger_width, pip_y],
                [mcp_x + finger_width, mcp_y]
            ], np.int32)
            # draw green polygon
            cv2.polylines(frame, [points], True, (0, 255, 0), 2)
    else:
        
        paint_points=np.delete(paint_points, slice(None), axis=0)
    cv2.imshow("Index finger Detection", frame)
    out.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()
